##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Create a TFX pipeline using templates with Local orchestrator

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template_local"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/template_local.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/template_local.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
</table></div>

## 시작하기

This document will provide instructions to create a TensorFlow Extended (TFX) pipeline using *templates* which are provided with TFX Python package. Most of instructions are Linux shell commands, and corresponding Jupyter Notebook code cells which invoke those commands using `!` are provided.

You will build a pipeline using [Taxi Trips dataset](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) released by the City of Chicago. We strongly encourage you to try to build your own pipeline using your dataset by utilizing this pipeline as a baseline.

We will build a pipeline which runs on local environment. If you are interested in using Kubeflow orchestrator on Google Cloud, please see [TFX on Cloud AI Platform Pipelines tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

## 전제 조건

- Linux / MacOS
- Python &gt;= 3.5.3

You can get all prerequisites easily by [running this notebook on Google Colab](https://colab.sandbox.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/template_local.ipynb).


## 1단계. 환경 설정

**Throughout this document, we will present commands twice. Once as a copy-and-paste-ready shell command, once as a jupyter notebook cell. If you are using Colab, just skip shell script block and execute notebook cells.**

You should prepare a development environment to build a pipeline.

Install `tfx` python package. We recommend use of `virtualenv` in the local environment. You can use following shell script snippet to set up your environment.

```sh
# Create a virtualenv for tfx.
virtualenv -p python3 venv
source venv/bin/activate
# Install python packages.
python -m pip install --upgrade "tfx<2"
```

If you are using colab:


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "tfx<2"

NOTE: There might be some errors during package installation. For example,

>ERROR: some-package 0.some_version.1 has requirement other-package!=2.0.,&lt;3,&gt;=1.15, but you'll have other-package 2.0.0 which is incompatible.

Please ignore these errors at this moment.

In [ ]:
# Set `PATH` to include user python binary directory.
HOME=%env HOME
PATH=%env PATH
%env PATH={PATH}:{HOME}/.local/bin

Let's check the version of TFX.
```bash
python -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"
```

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

이제 파이프라인을 생성할 수 있습니다.

## Step 2. Copy predefined template to your project directory.

이 단계에서는 미리 정의된 템플릿에서 추가 파일을 복사하여 작업 파이프라인 프로젝트 디렉터리 및 파일을 생성하겠습니다.

아래 `PIPELINE_NAME`을 변경하여 파이프라인에 다른 이름을 지정할 수 있으며, 이는 파일이 저장되는 프로젝트 디렉터리 이름으로도 사용됩니다.

```bash
export PIPELINE_NAME="my_pipeline"
export PROJECT_DIR=~/tfx/${PIPELINE_NAME}
```

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Create a project directory under Colab content directory.
PROJECT_DIR=os.path.join(os.sep,"content",PIPELINE_NAME)

TFX에는 TFX Python 패키지가 있는 `taxi` 템플릿이 포함되어 있습니다. 분류 및 회귀 등의 포인트별 예측 문제를 해결하려는 경우 이 템플릿으로 시작할 수 있습니다.

`tfx template copy` CLI 명령은 미리 정의된 템플릿 파일을 프로젝트 디렉토리에 복사합니다.

```sh
tfx template copy \
   --pipeline_name="${PIPELINE_NAME}" \
   --destination_path="${PROJECT_DIR}" \
   --model=taxi
```

In [ ]:
!tfx template copy \
  --pipeline_name={PIPELINE_NAME} \
  --destination_path={PROJECT_DIR} \
  --model=taxi

이 노트북의 작업 디렉토리 컨텍스트를 프로젝트 디렉토리로 변경합니다.

```bash
cd ${PROJECT_DIR}
```

In [ ]:
%cd {PROJECT_DIR}

## Step 3. Browse your copied source files.

TFX 템플릿은 파이프라인의 출력을 분석하기 위해 Python 소스 코드, 샘플 데이터 및 주피터 노트북을 포함하여 파이프라인을 구축하기 위한 기본 스캐폴드 파일을 제공합니다. `taxi` 템플릿은 <a>Airflow 튜토리얼</a>과 동일한 <em>Chicago Taxi</em> 데이터세트 및 ML 모델을 사용합니다.

In Google Colab, you can browse files by clicking a folder icon on the left. Files should be copied under the project directoy, whose name is `my_pipeline` in this case. You can click directory names to see the content of the directory, and double-click file names to open them.

다음은 각 Python 파일에 대한 간략한 소개입니다.

- `pipeline` -이 디렉토리는 파이프라인을 정의합니다.
    - `configs.py` — 파이프라인 러너의 공통 상수를 정의합니다.
    - `pipeline.py` — TFX 컴포넌트 및 파이프라인을 정의합니다.
- `models` - 이 디렉토리는 ML 모델을 정의합니다.
    - `features.py`, `features_test.py` — 모델의 특성을 정의합니다.
    - `preprocessing.py`, `preprocessing_test.py` — `tf::Transform`을 사용하여 전처리 작업을 정의합니다.
    - `estimator` - 이 디렉토리는 Estimator 기반 모델을 포함합니다.
        - `constants.py` — 모델 상수를 정의합니다.
        - `model.py`, `model_test.py` — TF estimator를 사용하여 DNN 모델을 정의합니다.
    - `keras` -- 이 디렉토리는 Keras 기반 모델을 포함합니다.
        - `constants.py` — 모델 상수를 정의합니다.
        - `model.py`, `model_test.py` — Keras를 사용하여 DNN 모델을 정의합니다.
- `local_runner.py`, `kubeflow_runner.py` — 각 오케스트레이션 엔진의 러너를 정의합니다.


이름에 `_test.py`가 포함된 일부 파일이 있음을 알 수 있습니다. 이는 파이프라인의 단위 테스트이며 자체 파이프라인을 구현할 때 더 많은 단위 테스트를 추가하는 것이 좋습니다. `-m` 플래그와 함께 테스트 파일의 모듈 이름을 제공하여 단위 테스트를 실행할 수 있습니다. 일반적으로 `.py` 확장자를 삭제하고 `/`를 `.`로 대체하여 모듈 이름을 얻을 수 있습니다. 예를 들면 다음과 같습니다.

```bash
python -m models.features_test
```

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## 4단계. 첫 번째 TFX 파이프라인 실행

You can create a pipeline using `pipeline create` command.

```bash
tfx pipeline create --engine=local --pipeline_path=local_runner.py
```


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

Then, you can run the created pipeline using `run create` command.
```sh
tfx run create --engine=local --pipeline_name="${PIPELINE_NAME}"
```

In [ ]:
!tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

If successful, you'll see `Component CsvExampleGen is finished.` When you copy the template, only one component, CsvExampleGen, is included in the pipeline.

## 5단계. 데이터 검증을 위한 구성 요소 추가

`StatisticsGen`, `SchemaGen`, `ExampleValidator`을 포함하여 데이터 검증을 위한 구성 요소를 추가합니다. 데이터 검증에 대해 자세히 알고 싶으면 [Tensorflow 데이터 검증 시작하기](https://www.tensorflow.org/tfx/data_validation/get_started)를 참조하세요.

We will modify copied pipeline definition in `pipeline/pipeline.py`. If you are working on your local environment, use your favorite editor to edit the file. If you are working on Google Colab,

> **Click folder icon on the left to open `Files` view**.

> **Click `my_pipeline` to open the directory and click `pipeline` directory to open and double-click `pipeline.py` to open the file**.

> Find and uncomment the 3 lines which add `StatisticsGen`, `SchemaGen`, and `ExampleValidator` to the pipeline. (Tip: find comments containing `TODO(step 5):`).

> Your change will be saved automatically in a few seconds. Make sure that the `*` mark in front of the `pipeline.py` disappeared in the tab title. **There is no save button or shortcut for the file editor in Colab. Python files in file editor can be saved to the runtime environment even in `playground` mode.**

이제 수정된 파이프라인 정의로 기존 파이프라인을 업데이트해야 합니다. `tfx pipeline update` 명령을 사용하여 파이프라인을 업데이트한 다음 `tfx run create` 명령을 사용하여 업데이트된 파이프라인의 새 실행을 생성합니다.

```sh
# Update the pipeline
tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```


In [ ]:
# Update the pipeline
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
# You can run the pipeline the same way.
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

You should be able to see the output log from the added components. Our pipeline creates output artifacts in `tfx_pipeline_output/my_pipeline` directory.

## 6단계. 훈련용 구성 요소 추가

이 단계에서는 `Transform`, `Trainer`, `Resolver`, `Evaluator`, `Pusher`를 포함하여 훈련 및 모델 검증을 위한 구성 요소를 추가합니다.

> **Open `pipeline/pipeline.py`**. Find and uncomment 5 lines which add `Transform`, `Trainer`, `Resolver`, `Evaluator` and `Pusher` to the pipeline. (Tip: find `TODO(step 6):`)

이전처럼 이제 수정된 파이프라인 정의로 기존 파이프라인을 업데이트해야 합니다. 지침은 5단계와 동일합니다. `tfx pipeline update`를 사용하여 파이프라인을 업데이트하고 `tfx run create`를 사용하여 실행을 생성합니다.

```sh
tfx pipeline update --engine=local --pipeline_path=local_runner.py
tfx run create --engine local --pipeline_name "${PIPELINE_NAME}"
```

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

When this execution run finishes successfully, you have now created and run your first TFX pipeline using Local orchestrator!

**참고:** 파이프라인 실행을 생성할 때마다 입력 및 매개변수가 변경되지 않은 경우에도 모든 구성 요소가 계속해서 실행될 수 있습니다. 해당 실행은 파이프라인 캐싱으로 건너뛰어 시간과 리소스를 절약할 수 있습니다. `pipeline.py`의 `Pipeline` 객체에 `enable_cache=True`를 지정하여 캐싱을 활성화할 수 있습니다.


## Step 7. (*Optional*) Try BigQueryExampleGen.

<a>BigQuery</a>는 확장성이 뛰어나며 비용 효율적인 서버리스 클라우드 데이터 웨어하우스입니다. BigQuery는 TFX에서 훈련 예제 소스로 사용할 수 있습니다. 이 단계에서는 <code>BigQueryExampleGen</code>를 파이프라인에 추가하겠습니다.

You need a [Google Cloud Platform](https://cloud.google.com/gcp/getting-started) account to use BigQuery. Please prepare a GCP project.

Login to your project using colab auth library or `gcloud` utility.

```sh
# You need `gcloud` tool to login in local shell environment.
gcloud auth login
```

In [ ]:
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')


You should specify your GCP project name to access BigQuery resources using TFX. Set `GOOGLE_CLOUD_PROJECT` environment variable to your project name.

```sh
export GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE
```

In [ ]:
# Set your project name below.
# WARNING! ENTER your project name before running this cell.
%env GOOGLE_CLOUD_PROJECT=YOUR_PROJECT_NAME_HERE

> **Open `pipeline/pipeline.py`**. Comment out `CsvExampleGen` and uncomment the line which create an instance of `BigQueryExampleGen`. You also need to uncomment `query` argument of the `create_pipeline` function.

We need to specify which GCP project to use for BigQuery again, and this is done by setting `--project` in `beam_pipeline_args` when creating a pipeline.

> **Open `pipeline/configs.py`**. Uncomment the definition of `BIG_QUERY__WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` and `BIG_QUERY_QUERY`. You should replace the project id and the region value in this file with the correct values for your GCP project.

> **Open `local_runner.py`**. Uncomment two arguments, `query` and `beam_pipeline_args`, for create_pipeline() method.

Now the pipeline is ready to use BigQuery as an example source. Update the pipeline and create a run as we did in step 5 and 6.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py
!tfx run create --engine local --pipeline_name {PIPELINE_NAME}

## What's next: Ingest YOUR data to the pipeline.

Chicago Taxi 데이터세트를 사용하여 모델의 파이프라인을 만들었습니다. 이제 데이터를 파이프라인에 입력하겠습니다.

데이터는 GCS 또는 BigQuery를 포함하여 파이프라인이 액세스할 수 있는 모든 곳에 저장할 수 있습니다. 파이프라인 정의를 수정해야 데이터에 액세스할 수 있습니다.

1. 데이터가 파일에 저장된 경우 `kubeflow_runner.py` 또는 `local_runner.py`에서 `DATA_PATH`를 수정하고 해당 파일 위치로 설정합니다. 데이터가 BigQuery에 저장되어 있는 경우, `pipeline/configs.py`의 `BIG_QUERY_QUERY`를 수정하여 데이터를 올바로 쿼리할 수 있습니다.
2. `models/features.py`에서 특성을 추가합니다.
3. `models/preprocessing.py`를 수정하여 [훈련에 적합하게 입력 데이터로 변환](https://www.tensorflow.org/tfx/guide/transform)합니다.
4. `models/keras/model.py` 및 `models/keras/constants.py`를 수정하여 [ML 모델을 설명](https://www.tensorflow.org/tfx/guide/trainer)합니다.
    - Estimator 기반 모델도 사용할 수 있습니다. `pipeline/configs.py`에서 `RUN_FN` 상수를 `models.estimator.model.run_fn`으로 변경합니다.

자세한 소개는 [Trainer 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/trainer)를 참조하세요.